<a href="https://colab.research.google.com/github/vgaby23/Shopee-Code-League/blob/master/Logistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import csv
print(open('/content/delivery_orders_march.csv'))

df = pd.read_csv('/content/delivery_orders_march.csv', 
              sep = ',', 
              encoding = 'utf-8')
df.head()

<_io.TextIOWrapper name='/content/delivery_orders_march.csv' mode='r' encoding='UTF-8'>


,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,1.583138e+09,1.583385e+09,NaN,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,1.583310e+09,1.583463e+09,1.583799e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,1.583306e+09,1.583460e+09,NaN,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,1.583419e+09,1.583556e+09,NaN,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,1.583318e+09,1.583480e+09,NaN,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...


In [ ]:
df.dtypes
df.dropna( subset = ['buyeraddress', 'selleraddress'], inplace = True )
df.isnull().sum()

orderid                     0
pick                        0
1st_deliver_attempt         0
2nd_deliver_attempt    113196
buyeraddress                0
selleraddress               0
dtype: int64

In [ ]:
#lower buyeraddress and selleraddress string and city only 
df['buyeraddress'] = df['buyeraddress'].apply(lambda x: x.split()[-1]).str.lower()
df['selleraddress'] = df['selleraddress'].apply(lambda x: x.split()[-1]).str.lower()
df.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,1.583138e+09,1.583385e+09,NaN,manila,manila
1,2219624609,1.583310e+09,1.583463e+09,1.583799e+09,manila,manila
2,2220979489,1.583306e+09,1.583460e+09,NaN,manila,manila
3,2221066352,1.583419e+09,1.583556e+09,NaN,manila,manila
4,2222478803,1.583318e+09,1.583480e+09,NaN,luzon,manila


In [ ]:
#change epoch to date time series
df['2nd_deliver_attempt'].fillna(value = 0, inplace = True )
#change timezone to GMT+8
df[['pick', '1st_deliver_attempt', '2nd_deliver_attempt']] += (8*60*60)
df['pick'] = pd.to_datetime(df['pick'], unit = 's').dt.date
df['1st_deliver_attempt'] = pd.to_datetime(df['1st_deliver_attempt'], unit = 's').dt.date
df['2nd_deliver_attempt'] = pd.to_datetime(df['2nd_deliver_attempt'], unit = 's').dt.date
df.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,2020-03-02,2020-03-05,1970-01-01,manila,manila
1,2219624609,2020-03-04,2020-03-06,2020-03-10,manila,manila
2,2220979489,2020-03-04,2020-03-06,1970-01-01,manila,manila
3,2221066352,2020-03-05,2020-03-07,1970-01-01,manila,manila
4,2222478803,2020-03-04,2020-03-06,1970-01-01,luzon,manila


In [ ]:
#determining the duration of the delivery
days = []

for x,y in df[['buyeraddress', 'selleraddress']].itertuples(index = False):
  if (x == 'manila' and y == 'manila'):
    days.append(3)
  elif ((x == 'luzon' and y =='luzon') or (x == 'luzon' and y =='manila') or (x == 'manila' and y == 'luzon')):
    days.append(5)
  else :
    days.append(7)

df['delivery_duration_limit'] = days
df.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,delivery_duration_limit
0,2215676524,2020-03-02,2020-03-05,1970-01-01,manila,manila,3
1,2219624609,2020-03-04,2020-03-06,2020-03-10,manila,manila,3
2,2220979489,2020-03-04,2020-03-06,1970-01-01,manila,manila,3
3,2221066352,2020-03-05,2020-03-07,1970-01-01,manila,manila,3
4,2222478803,2020-03-04,2020-03-06,1970-01-01,luzon,manila,5


In [ ]:
import numpy as np
#counting days between pick and 1st_deliver_attempt or 2nd_deliver_attempt
holiday = ['2020-03-08', '2020-03-25', '2020-03-30', '2020-03-31']
df['1st_duration'] = np.busday_count(df['pick'], df['1st_deliver_attempt'], weekmask = '1111110', holidays = holiday)
df['2nd_duration'] = np.busday_count(df['1st_deliver_attempt'], df['2nd_deliver_attempt'], weekmask = '1111110', holidays = holiday)
df.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,delivery_duration_limit,1st_duration,2nd_duration
0,2215676524,2020-03-02,2020-03-05,1970-01-01,manila,manila,3,3,-15708
1,2219624609,2020-03-04,2020-03-06,2020-03-10,manila,manila,3,2,3
2,2220979489,2020-03-04,2020-03-06,1970-01-01,manila,manila,3,2,-15709
3,2221066352,2020-03-05,2020-03-07,1970-01-01,manila,manila,3,2,-15710
4,2222478803,2020-03-04,2020-03-06,1970-01-01,luzon,manila,5,2,-15709


In [ ]:
#determining late and on-time delivery 
df['is_late'] = (df['1st_duration'] > df['delivery_duration_limit']) | (df['2nd_duration'] > 3 )
df['is_late'] = df['is_late'].astype(int)
df.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,delivery_duration_limit,1st_duration,2nd_duration,is_late
0,2215676524,2020-03-02,2020-03-05,1970-01-01,manila,manila,3,3,-15708,0
1,2219624609,2020-03-04,2020-03-06,2020-03-10,manila,manila,3,2,3,0
2,2220979489,2020-03-04,2020-03-06,1970-01-01,manila,manila,3,2,-15709,0
3,2221066352,2020-03-05,2020-03-07,1970-01-01,manila,manila,3,2,-15710,0
4,2222478803,2020-03-04,2020-03-06,1970-01-01,luzon,manila,5,2,-15709,0


In [ ]:
#Lateness orderid data
d = {'order_id' : df['orderid'], 'is_late': df['is_late']}
data = pd.DataFrame(data = d)
data.head(20)

,order_id,is_late
0,2215676524,0
1,2219624609,0
2,2220979489,0
3,2221066352,0
4,2222478803,0
5,2222597288,0
6,2222738456,0
7,2224695304,1
8,2224704587,0
9,2225138267,0
